In [ ]:
import numpy as np
import pickle
import pandas as pd
import os
from os.path import join
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from scipy import stats
import xgboost as xgb

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl


# A. rxnfp reproduction


With previous hyperparameter optimization

In [ ]:
data_train =   pd.read_pickle(join("data" , "kcat_data", "splits", "modified_splits", "train_df_kcat_rxnfp.pkl"))
data_test  =   pd.read_pickle(join("data" , "kcat_data", "splits", "modified_splits", "test_df_kcat_rxnfp.pkl"))

In [ ]:
data_train =   pd.read_pickle("/home/yvan/Desktop/BERT_ft_enzymatic_reactions/data_new/kcat_data/splits/train_df_kcat_rxnfp_repro2.pkl")
data_test  =   pd.read_pickle("/home/yvan/Desktop/BERT_ft_enzymatic_reactions/data_new/kcat_data/splits/test_df_kcat_rxnfp_repro2.pkl")

In [ ]:
train_X = np.array(list(data_train["rxnfp"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["rxnfp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

train_indices = list(np.load(join( "data","kcat_data","splits","CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join( "data","kcat_data","splits","CV_test_indices.npy"), allow_pickle = True))

In [ ]:
param = {
    "learning_rate": 0.04933913947630953,
    "max_delta_step": 2.548116839939456,
    "max_depth": 7.51666173394416,
    "min_child_weight": 4.517403135970991,
    "num_rounds": 146.52346851005683,
    "reg_alpha": 1.989495668303869,
    "reg_lambda": 0.5876902849399113
}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []
y_valid_pred_RXNFP = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_RXNFP.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

[0.5568630444269154, 0.5857452035180237, 0.5571659855018288, 0.5790982861324072, 0.5779793769211048]
[0.9068863712875874, 0.8305728110243333, 0.9215306191499872, 1.0675509913915044, 1.0070897770806566]
[0.3098495915724485, 0.3409420332449373, 0.3034387403370621, 0.33424210620890726, 0.33394881062574955]


In [ ]:
[0.3098495915724485, 0.3409420332449373, 0.3034387403370621, 0.33424210620890726, 0.33394881062574955]

In [ ]:
np.save(join( "data", "training_results", "Pearson_CV_xgboost_rxnfp.npy"), np.array(Pearson))
np.save(join( "data", "training_results", "MSE_CV_xgboost_rxnfp.npy"), np.array(MSE))
np.save(join(  "data", "training_results", "R2_CV_xgboost_rxnfp.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))



0.618 0.89 0.379


In [ ]:

np.save(join( "data", "training_results", "y_test_pred_xgboost_rxnfp.npy"), bst.predict(dtest))
np.save(join( "data", "training_results", "y_test_true_xgboost_rxnfp.npy"), test_Y)

In [ ]:
#y_valid_pred_RXNFP
y_test_pred_rxnfp = y_test_pred